In [1]:
#Importing necessary libraries
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import SimpleRNN, Activation, Dense, Dropout, Input, Embedding, LSTM
from keras.models import Model
from keras.metrics import Precision
from keras.utils import pad_sequences

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Input and read in spam dataset, check head of dataset
df = pd.read_csv('/content/drive/Shareddrives/CS263 Project/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
#Create input vectors and match to correct column
X_input = df.v2
Y_input = df.v1

#Use label encoder to encode target labels (not input values)
labels = preprocessing.LabelEncoder()

Y = labels.fit_transform(Y_input)
Y = Y.reshape(-1,1)

In [6]:
#Split dataset into training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X_input,Y,test_size=0.2)

In [7]:
#Tokenize columns
tok = Tokenizer(num_words=300)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=300)

In [8]:
inputs = Input(name='inputs',shape=[300])
layer = Embedding(300,50,input_length=300)(inputs)
layer = SimpleRNN(256)(layer)
layer = Dense(256,name='FC1',activation='relu')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)

In [9]:
#Summarize and then compile model
model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy', 'Precision', 'Recall'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 50)           15000     
                                                                 
 simple_rnn (SimpleRNN)      (None, 256)               78592     
                                                                 
 FC1 (Dense)                 (None, 256)               65792     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [10]:
#Fit model on training data
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 24s 771ms/step - loss: 0.4698 - accuracy: 0.8325 - precision: 0.1329 - recall: 0.0386 - val_loss: 0.3612 - val_accuracy: 0.8823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
28/28 [==============================] - 20s 701ms/step - loss: 0.5224 - accuracy: 0.8222 - precision: 0.1553 - recall: 0.0650 - val_loss: 0.3653 - val_accuracy: 0.8823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
28/28 [==============================] - 21s 760ms/step - loss: 0.4602 - accuracy: 0.8429 - precision: 0.1964 - recall: 0.0447 - val_loss: 0.3684 - val_accuracy: 0.8823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
28/28 [==============================] - 20s 736ms/step - loss: 0.4840 - accuracy: 0.8412 - precision: 0.0978 - recall: 0.0183 - val_loss: 0.5424 - val_accuracy: 0.8823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10
28/28 [==============================] - 21s 745ms/step -

In [11]:
#check accuracy and precision on test data
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=300)

In [12]:
#Report final test accuracy
final_test_accuracy = model.evaluate(test_sequences_matrix,Y_test)

35/35 [==============================] - 3s 91ms/step - loss: 0.1820 - accuracy: 0.9363 - precision: 0.8990 - recall: 0.5933
